In [ ]:
import re

def transform_query(source_query):
    # Normalize query: remove extra spaces, newlines, unify case for parsing
    query = re.sub(r'\s+', ' ', source_query.strip())

    # Define the transformation rules
    rules = {
        "CRMD_ORDERADM_H": "CRMS4D_SERV_H",
        "CRMD_LINK": None,
        "CRMD_ORGMAN": None,
    }

    field_mapping = {
      "GUID": "HEADER_GUID",
      "SALES_ORG": "SALES_ORG",
      "ARCHIVING_FLAG": "ARCHIVING_FLAG",
      "AUTH_SCOPE": "AUTH_SCOPE",
      "BTX_CLASS": "BTX_CLASS",
      "CHANGED_AT": "CHANGED_AT_H",
      "CHANGED_BY": "CHANGED_BY_H",
      "CREATED_AT": "CREATED_AT_H",
      "CREATED_BY": "CREATED_BY_H",
      "DESCRIPTION": "DESCRIPTION_H",
      "DESCR_LANGUAGE": "DESCR_LANGUAGE",
      "HEAD_CHANGED_AT": "HEAD_CHANGED_AT",
      "OBJECT_ID": "OBJECT_ID",
      "OBJECT_ID_OK": "OBJECT_ID_OK",
      "OBJECT_TYPE": "OBJTYPE_H",
      "POSTING_DATE": "POSTING_DATE",
      "KNUMV": "PRICING_DOCUMENT",
      "KALSM": "PRICING_PROCEDURE",
      "PROCESS_TYPE": "PROCESS_TYPE",
      "TEMPLATE_TYPE": "TEMPLATE_TYPE",
      "VERIFY_DATE": "VERIFY_DATE"
    }

    # 1. Detect if SINGLE is present
    single = bool(re.search(r'\bSINGLE\b', query, re.IGNORECASE))

    # 2. Detect FOR ALL ENTRIES
    fae_match = re.search(r'FOR ALL ENTRIES IN (\w+)', query, re.IGNORECASE)
    fae_clause = ''
    if fae_match:
        fae_clause = f'FOR ALL ENTRIES IN {fae_match.group(1).upper()}'

    # 3. Detect UP TO ... ROWS
    up_to_match = re.search(r'UP TO (\w+) ROWS', query, re.IGNORECASE)
    up_to_clause = ''
    if up_to_match:
        up_to_clause = f'UP TO {up_to_match.group(1)} ROWS'

    # 4. Extract fields: from SELECT ... FROM or FIELDS ...
    fields = []
    select_fields_match = re.search(r'SELECT (.*?) FROM', query, re.IGNORECASE)
    if select_fields_match:
        fields_str = select_fields_match.group(1).strip()
        # If no comma, split by spaces (for queries like SELECT A~GUID C~SALES_ORG ...)
        if ',' in fields_str:
            fields = [f.strip() for f in fields_str.split(',')]
        else:
            fields = [f.strip() for f in fields_str.split() if f.strip()]
    else:
        # Try FIELDS ...
        fields_match = re.search(r'FIELDS (.*?)(WHERE|INTO|$)', query, re.IGNORECASE)
        if fields_match:
            fields_str = fields_match.group(1).strip().rstrip(',')
            if ',' in fields_str:
                fields = [f.strip() for f in fields_str.split(',')]
            else:
                fields = [f.strip() for f in fields_str.split() if f.strip()]

    # Remove 'SINGLE' from fields if present (case-insensitive)
    fields = [f for f in fields if f.strip().upper() != 'SINGLE']

    # 5. Transform fields
    transformed_fields = []
    for field in fields:
        # Remove any @DATA(...) or @ prefix for field mapping only
        field_for_map = re.sub(r'@DATA\((.*?)\)', r'\1', field)
        field_for_map = field_for_map.lstrip('@')
        # Handle aliasing (A~GUID, C~SALES_ORG, etc.)
        if '~' in field_for_map:
            alias, fname = field_for_map.split('~', 1)
        else:
            fname = field_for_map
        fname = fname.strip().upper()
        mapped = field_mapping.get(fname, fname)
        transformed_fields.append(f'{mapped} AS {fname}')

    # 6. Extract INTO clause (INTO TABLE ... or INTO ...), preserve @DATA(...) and @
    into_clause = ''
    into_match = re.search(r'INTO\s+(TABLE\s+)?((@?DATA\([^)]+\))|(@?\w+))', query, re.IGNORECASE)
    if into_match:
        into_var = into_match.group(2)
        if into_match.group(1): # TABLE present
            into_clause = f'INTO TABLE {into_var}'
        else:
            into_clause = f'INTO {into_var}'

    # 7. Extract WHERE clause, preserve @
    where_clause = ''
    where_match = re.search(r'WHERE (.*?)(FOR ALL ENTRIES|INTO|UP TO|$)', query, re.IGNORECASE)
    if where_match:
        where_cond = where_match.group(1).strip()
        # Replace ~ in WHERE (A~GUID EQ LV_GUID => HEADER_GUID EQ LV_GUID)
        def replace_tilde(m):
            return field_mapping.get(m.group(2).upper(), m.group(2).upper())
        where_cond = re.sub(r'(\w+)~(\w+)', replace_tilde, where_cond)
        # DO NOT remove @ prefix in WHERE
        where_clause = f'WHERE {where_cond}'

    # 8. Compose the final query
    parts = []
    if single:
        parts.append('SELECT SINGLE')
    else:
        parts.append('SELECT')
    parts.append(', '.join(transformed_fields)) # Use comma, not space, between fields
    parts.append(f'FROM {rules["CRMD_ORDERADM_H"]}')
    if into_clause:
        parts.append(into_clause)
    if fae_clause:
        parts.append(fae_clause)
    if up_to_clause:
        parts.append(up_to_clause)
    if where_clause:
        parts.append(where_clause)

    transformed_query = ' '.join(parts)
    return transformed_query

# Test the function with all provided queries and print results
test_queries = [
    # R1
    '''SELECT A~GUID C~SALES_ORG, KNUMV FROM CRMD_ORDERADM_H AS A INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID INNER JOIN CRMD_ORGMAN AS C ON C~GUID = B~GUID_SET INTO TABLE LT_ORG WHERE A~GUID EQ LV_GUID''',
    # R2
    '''SELECT A~GUID, C~SALES_ORG FROM CRMD_ORDERADM_H AS A INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID INNER JOIN CRMD_ORGMAN AS C ON C~GUID = B~GUID_SET INTO TABLE @DATA(LT_ORG) WHERE A~GUID EQ @LV_GUID''',
    # R3
    '''SELECT A~GUID, C~SALES_ORG FROM CRMD_ORDERADM_H AS A INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID INNER JOIN CRMD_ORGMAN AS C ON C~GUID = B~GUID_SET INTO TABLE @LT_ORG WHERE A~GUID EQ @LV_GUID''',
    # R4
    '''SELECT FROM CRMD_ORDERADM_H AS A INNER JOIN CRMD_LINK AS B ON B~GUID_HI  = A~GUID INNER JOIN CRMD_ORGMAN AS C ON C~GUID  = B~GUID_SET FIELDS A~GUID,  C~SALES_ORG WHERE  A~GUID EQ  @LV_GUID INTO TABLE @LT_ORG''',
    # R5
    '''SELECT A~GUID C~SALES_ORG FROM CRMD_ORDERADM_H AS A INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID INNER JOIN CRMD_ORGMAN AS C ON C~GUID = B~GUID_SET FOR ALL ENTRIES IN LT_GUID INTO TABLE LT_ORG WHERE A~GUID EQ LT_GUID-GUID''',
    # R8
    '''SELECT SINGLE A~GUID C~SALES_ORG FROM CRMD_ORDERADM_H AS A INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID INNER JOIN CRMD_ORGMAN AS C ON C~GUID = B~GUID_SET INTO LS_ORG WHERE A~GUID EQ LV_GUID''',
    # R9
    '''SELECT SINGLE A~GUID C~SALES_ORG FROM CRMD_ORDERADM_H AS A INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID INNER JOIN CRMD_ORGMAN AS C ON C~GUID = B~GUID_SET INTO TABLE LT_ORG UP TO XXXX ROWS WHERE A~GUID EQ LV_GUID''',
    # R10
    '''SELECT A~GUID C~SALES_ORG FROM CRMD_ORDERADM_H AS A INNER JOIN CRMD_LINK AS B  ON B~GUID_HI  = A~GUID INNER JOIN CRMD_ORGMAN AS C ON C~GUID  = B~GUID_SET INTO  LS_ORG WHERE  A~GUID EQ   LV_GUID.''',
]

for i, q in enumerate(test_queries, 1):
    print(f'R{i}:', transform_query(q))


    

R1: SELECT GUID C~SALES_ORG AS GUID C~SALES_ORG, PRICING_DOCUMENT AS KNUMV FROM CRMS4D_SERV_H INTO TABLE LT_ORG WHERE HEADER_GUID EQ LV_GUID
R2: SELECT HEADER_GUID AS GUID, SALES_ORG AS SALES_ORG FROM CRMS4D_SERV_H INTO TABLE @DATA(LT_ORG) WHERE HEADER_GUID EQ @LV_GUID
R3: SELECT HEADER_GUID AS GUID, SALES_ORG AS SALES_ORG FROM CRMS4D_SERV_H INTO TABLE @LT_ORG WHERE HEADER_GUID EQ @LV_GUID
R4: SELECT HEADER_GUID AS GUID, SALES_ORG AS SALES_ORG FROM CRMS4D_SERV_H INTO TABLE @LT_ORG WHERE HEADER_GUID EQ @LV_GUID
R5: SELECT HEADER_GUID AS GUID, SALES_ORG AS SALES_ORG FROM CRMS4D_SERV_H INTO TABLE LT_ORG FOR ALL ENTRIES IN LT_GUID WHERE HEADER_GUID EQ LT_GUID-GUID
R6: SELECT SINGLE HEADER_GUID AS GUID, SALES_ORG AS SALES_ORG FROM CRMS4D_SERV_H INTO LS_ORG WHERE HEADER_GUID EQ LV_GUID
R7: SELECT SINGLE HEADER_GUID AS GUID, SALES_ORG AS SALES_ORG FROM CRMS4D_SERV_H INTO TABLE LT_ORG UP TO XXXX ROWS WHERE HEADER_GUID EQ LV_GUID
R8: SELECT HEADER_GUID AS GUID, SALES_ORG AS SALES_ORG FROM CRMS4

In [ ]:
import pandas as pd
import requests
import re

# Download the Excel file from GitHub
excel_url = "https://github.com/vijayyekbote/CodeRemediation/blob/main/Select%20Queries.xlsx"
excel_file = "Select_Queries.xlsx"
with open(excel_file, "wb") as f:
    f.write(requests.get(excel_url).content)

# Load mapping from Excel
df_map = pd.read_excel(excel_file)
table_mapping = dict(zip(df_map['CRM_TABNAME'], df_map['S4_TABNAME']))
field_mapping = dict(zip(df_map['CRM_FIELDNAME'], df_map['S4_FIELDNAME']))

def transform_query(source_query):

    query = re.sub(r'\s+', ' ', source_query.strip())

    # 1. Detect if SINGLE is present
    single = bool(re.search(r'\bSINGLE\b', query, re.IGNORECASE))

    # 2. Detect FOR ALL ENTRIES
    fae_match = re.search(r'FOR ALL ENTRIES IN (\w+)', query, re.IGNORECASE)
    fae_clause = ''
    if fae_match:
        fae_clause = f'FOR ALL ENTRIES IN {fae_match.group(1).upper()}'

    # 3. Detect UP TO ... ROWS
    up_to_match = re.search(r'UP TO (\w+) ROWS', query, re.IGNORECASE)
    up_to_clause = ''
    if up_to_match:
        up_to_clause = f'UP TO {up_to_match.group(1)} ROWS'

    # 4. Extract fields
    fields = []
    select_fields_match = re.search(r'SELECT (.*?) FROM', query, re.IGNORECASE)
    if select_fields_match:
        fields_str = select_fields_match.group(1).strip()
        if ',' in fields_str:
            fields = [f.strip() for f in fields_str.split(',')]
        else:
            fields = [f.strip() for f in fields_str.split() if f.strip()]
    else:
        fields_match = re.search(r'FIELDS (.*?)(WHERE|INTO|$)', query, re.IGNORECASE)
        if fields_match:
            fields_str = fields_match.group(1).strip().rstrip(',')
            if ',' in fields_str:
                fields = [f.strip() for f in fields_str.split(',')]
            else:
                fields = [f.strip() for f in fields_str.split() if f.strip()]

    fields = [f for f in fields if f.strip().upper() != 'SINGLE']

    # 5. Transform fields
    transformed_fields = []
    for field in fields:
        field_for_map = re.sub(r'@DATA\((.*?)\)', r'\1', field)
        field_for_map = field_for_map.lstrip('@')
        if '~' in field_for_map:
            alias, fname = field_for_map.split('~', 1)
        else:
            fname = field_for_map
        fname = fname.strip().upper()
        mapped = field_mapping.get(fname, fname)
        transformed_fields.append(f'{mapped} AS {fname}')

    # 6. Extract INTO clause
    into_clause = ''
    into_match = re.search(r'INTO\s+(TABLE\s+)?((@?DATA\([^)]+\))|(@?\w+))', query, re.IGNORECASE)
    if into_match:
        into_var = into_match.group(2)
        if into_match.group(1):
            into_clause = f'INTO TABLE {into_var}'
        else:
            into_clause = f'INTO {into_var}'

    # 7. Extract WHERE clause
    where_clause = ''
    where_match = re.search(r'WHERE (.*?)(FOR ALL ENTRIES|INTO|UP TO|$)', query, re.IGNORECASE)
    if where_match:
        where_cond = where_match.group(1).strip()
        def replace_tilde(m):
            return field_mapping.get(m.group(2).upper(), m.group(2).upper())
        where_cond = re.sub(r'(\w+)~(\w+)', replace_tilde, where_cond)
        where_clause = f'WHERE {where_cond}'

    # 8. Compose the final query
    # Find main table name from FROM clause
    from_match = re.search(r'FROM (\w+)', query, re.IGNORECASE)
    main_table = from_match.group(1) if from_match else None
    s4_table = table_mapping.get(main_table, main_table)

    parts = []
    if single:
        parts.append('SELECT SINGLE')
    else:
        parts.append('SELECT')
    parts.append(', '.join(transformed_fields))
    parts.append(f'FROM {s4_table}')
    if into_clause:
        parts.append(into_clause)
    if fae_clause:
        parts.append(fae_clause)
    if up_to_clause:
        parts.append(up_to_clause)
    if where_clause:
        parts.append(where_clause)

    transformed_query = ' '.join(parts)
    return transformed_query

# Example usage:
print(transform_query(q))

NameError: name 'q' is not defined

In [3]:
import pandas as pd
import requests
import re

# Download the Excel file from GitHub
excel_url = "https://github.com/srinivashj1/code/raw/main/Select%20Queries%201.xlsx"
excel_file = "Select_Queries_1.xlsx"
with open(excel_file, "wb") as f:
    f.write(requests.get(excel_url).content)

# Load mapping from Excel
df_map = pd.read_excel(excel_file)
table_mapping = dict(zip(df_map['CRM_TABNAME'], df_map['S4_TABNAME']))
field_mapping = dict(zip(df_map['CRM_FIELDNAME'], df_map['S4_FIELDNAME']))

def transform_query(source_query):
    query = re.sub(r'\s+', ' ', source_query.strip())

    # Detect if SINGLE is present
    single = bool(re.search(r'\bSINGLE\b', query, re.IGNORECASE))

    # Detect FOR ALL ENTRIES
    fae_match = re.search(r'FOR ALL ENTRIES IN (\w+)', query, re.IGNORECASE)
    fae_clause = ''
    if fae_match:
        fae_clause = f'FOR ALL ENTRIES IN {fae_match.group(1).upper()}'

    # Detect UP TO ... ROWS
    up_to_match = re.search(r'UP TO (\w+) ROWS', query, re.IGNORECASE)
    up_to_clause = ''
    if up_to_match:
        up_to_clause = f'UP TO {up_to_match.group(1)} ROWS'

    # Extract fields
    fields = []
    select_fields_match = re.search(r'SELECT (.*?) FROM', query, re.IGNORECASE)
    if select_fields_match:
        fields_str = select_fields_match.group(1).strip()
        if ',' in fields_str:
            fields = [f.strip() for f in fields_str.split(',')]
        else:
            fields = [f.strip() for f in fields_str.split() if f.strip()]
    else:
        fields_match = re.search(r'FIELDS (.*?)(WHERE|INTO|$)', query, re.IGNORECASE)
        if fields_match:
            fields_str = fields_match.group(1).strip().rstrip(',')
            if ',' in fields_str:
                fields = [f.strip() for f in fields_str.split(',')]
            else:
                fields = [f.strip() for f in fields_str.split() if f.strip()]

    fields = [f for f in fields if f.strip().upper() != 'SINGLE']

    # Transform fields
    transformed_fields = []
    for field in fields:
        field_for_map = re.sub(r'@DATA\((.*?)\)', r'\1', field)
        field_for_map = field_for_map.lstrip('@')
        if '~' in field_for_map:
            alias, fname = field_for_map.split('~', 1)
        else:
            fname = field_for_map
        fname = fname.strip().upper()
        mapped = field_mapping.get(fname, fname)
        transformed_fields.append(f'{mapped} AS {fname}')

    # Extract INTO clause
    into_clause = ''
    into_match = re.search(r'INTO\s+(TABLE\s+)?((@?DATA\([^)]+\))|(@?\w+))', query, re.IGNORECASE)
    if into_match:
        into_var = into_match.group(2)
        if into_match.group(1):
            into_clause = f'INTO TABLE {into_var}'
        else:
            into_clause = f'INTO {into_var}'

    # Extract WHERE clause
    where_clause = ''
    where_match = re.search(r'WHERE (.*?)(FOR ALL ENTRIES|INTO|UP TO|$)', query, re.IGNORECASE)
    if where_match:
        where_cond = where_match.group(1).strip()
        def replace_tilde(m):
            return field_mapping.get(m.group(2).upper(), m.group(2).upper())
        where_cond = re.sub(r'(\w+)~(\w+)', replace_tilde, where_cond)
        where_clause = f'WHERE {where_cond}'

    # Compose the final query
    from_match = re.search(r'FROM (\w+)', query, re.IGNORECASE)
    main_table = from_match.group(1) if from_match else None
    s4_table = table_mapping.get(main_table, main_table)

    parts = []
    if single:
        parts.append('SELECT SINGLE')
    else:
        parts.append('SELECT')
    parts.append(', '.join(transformed_fields))
    parts.append(f'FROM {s4_table}')
    if into_clause:
        parts.append(into_clause)
    if fae_clause:
        parts.append(fae_clause)
    if up_to_clause:
        parts.append(up_to_clause)
    if where_clause:
        parts.append(where_clause)

    transformed_query = ' '.join(parts)
    return transformed_query

In [5]:
def transform_query(source_query):
    query = re.sub(r'\s+', ' ', source_query.strip())

    # List of tables whose INNER JOINs should be removed
    remove_join_tables = [
        'CRMD_LINK', 'CRMD_ORGMAN', 'CRMD_SALES', 'CRMD_BILLING', 'CRMD_PRICING', 'CRMD_PRICING_I'
    ]

    # Remove INNER JOINs for specified tables
    for tbl in remove_join_tables:
        # Remove patterns like: INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID
        pattern = rf'INNER JOIN {tbl} AS \w+ ON [^ ]+ = [^ ]+'
        query = re.sub(pattern, '', query, flags=re.IGNORECASE)

    # Detect if SINGLE is present
    single = bool(re.search(r'\bSINGLE\b', query, re.IGNORECASE))

    # Detect FOR ALL ENTRIES
    fae_match = re.search(r'FOR ALL ENTRIES IN (\w+)', query, re.IGNORECASE)
    fae_clause = ''
    if fae_match:
        fae_clause = f'FOR ALL ENTRIES IN {fae_match.group(1).upper()}'

    # Detect UP TO ... ROWS
    up_to_match = re.search(r'UP TO (\w+) ROWS', query, re.IGNORECASE)
    up_to_clause = ''
    if up_to_match:
        up_to_clause = f'UP TO {up_to_match.group(1)} ROWS'

    # Extract fields
    fields = []
    select_fields_match = re.search(r'SELECT (.*?) FROM', query, re.IGNORECASE)
    if select_fields_match:
        fields_str = select_fields_match.group(1).strip()
        if ',' in fields_str:
            fields = [f.strip() for f in fields_str.split(',')]
        else:
            fields = [f.strip() for f in fields_str.split() if f.strip()]
    else:
        fields_match = re.search(r'FIELDS (.*?)(WHERE|INTO|$)', query, re.IGNORECASE)
        if fields_match:
            fields_str = fields_match.group(1).strip().rstrip(',')
            if ',' in fields_str:
                fields = [f.strip() for f in fields_str.split(',')]
            else:
                fields = [f.strip() for f in fields_str.split() if f.strip()]

    fields = [f for f in fields if f.strip().upper() != 'SINGLE']

    # Transform fields
    transformed_fields = []
    for field in fields:
        field_for_map = re.sub(r'@DATA\((.*?)\)', r'\1', field)
        field_for_map = field_for_map.lstrip('@')
        if '~' in field_for_map:
            alias, fname = field_for_map.split('~', 1)
        else:
            fname = field_for_map
        fname = fname.strip().upper()
        mapped = field_mapping.get(fname, fname)
        transformed_fields.append(f'{mapped} AS {fname}')

    # Extract INTO clause
    into_clause = ''
    into_match = re.search(r'INTO\s+(TABLE\s+)?((@?DATA\([^)]+\))|(@?\w+))', query, re.IGNORECASE)
    if into_match:
        into_var = into_match.group(2)
        if into_match.group(1):
            into_clause = f'INTO TABLE {into_var}'
        else:
            into_clause = f'INTO {into_var}'

    # Extract WHERE clause
    where_clause = ''
    where_match = re.search(r'WHERE (.*?)(FOR ALL ENTRIES|INTO|UP TO|$)', query, re.IGNORECASE)
    if where_match:
        where_cond = where_match.group(1).strip()
        def replace_tilde(m):
            return field_mapping.get(m.group(2).upper(), m.group(2).upper())
        where_cond = re.sub(r'(\w+)~(\w+)', replace_tilde, where_cond)
        where_clause = f'WHERE {where_cond}'

    # Compose the final query
    from_match = re.search(r'FROM (\w+)', query, re.IGNORECASE)
    main_table = from_match.group(1) if from_match else None
    s4_table = table_mapping.get(main_table, main_table)

    parts = []
    if single:
        parts.append('SELECT SINGLE')
    else:
        parts.append('SELECT')
    parts.append(', '.join(transformed_fields))
    parts.append(f'FROM {s4_table}')
    if into_clause:
        parts.append(into_clause)
    if fae_clause:
        parts.append(fae_clause)
    if up_to_clause:
        parts.append(up_to_clause)
    if where_clause:
        parts.append(where_clause)

    transformed_query = ' '.join(parts)
    return transformed_query

In [6]:
seq = """SELECT A~GUID,
                  C~SALES_ORG
FROM CRMD_ORDERADM_I AS A
INNER JOIN CRMD_LINK AS B  ON B~GUID_HI  = A~GUID
INNER JOIN CRMD_ORGMAN AS C ON C~GUID  = B~GUID_SET
INTO TABLE @LT_ORG
WHERE  A~GUID EQ  @LV_GUID"""

print(transform_query(seq))

SELECT ITEM_GUID_CHAR AS GUID, SALES_ORG AS SALES_ORG FROM CRMS4D_SERV_I INTO TABLE @LT_ORG WHERE ITEM_GUID_CHAR EQ @LV_GUID
